In [8]:
%pip install openshift-client
import os
import openshift as oc
from jinja2 import Template
import time
import shutil
run_id = "6"
project = "fmv2"

os.environ['OPENSHIFT_CLIENT_PYTHON_DEFAULT_OC_PATH'] = '/tmp/oc'


model_name = 'sademo'
model_version = "6"# os.environ["MODEL_VERSION"]
build_name = f"seldon-model-{model_name}-v{model_version}"


print("Start OCP things...")
#print('OpenShift server version: {}'.format(oc.get_server_version()))
token = os.environ["OPENSHIFT_LOGIN_TOKEN"]
server = os.environ["OPENSHIFT_LOGIN_SERVER"]
# print(token)
# print(server)




#build from source Docker file
with oc.api_server(server):
    with oc.token(token):
        with oc.project(project), oc.timeout(10*60):
            print('OpenShift client version: {}'.format(oc.get_client_version()))
            #print('OpenShift server version: {}'.format(oc.get_server_version()))

            build_config = oc.selector(f"bc/{build_name}").count_existing() #.object
            print(oc.get_project_name())
            print(build_config)
            if build_config == 0:
                oc.new_build(["--strategy", "docker", "--binary", "--docker-image", "registry.access.redhat.com/ubi8/python-38:1-71", "--name", build_name ])
            else:
                build_details = oc.selector(f"bc/{build_name}").object()
                print(build_details.as_json())
                
            print("Starting Build and Wiating.....")
            build_exec = oc.start_build([build_name, "--from-dir", ".", "--follow", "--build-loglevel", "10"])
            print("Build Finished")
            status = build_exec.status()
            print(status)
            for k, v in oc.selector([f"bc/{build_name}"]).logs(tail=500).items():
                print('Build Log: {}\n{}\n\n'.format(k, v))

            #seldon_deploy = oc.selector(f"SeldonDeployment/{build_name}").count_existing()
            #experiment_id = mlflow.get_run(run_id).info.experiment_id
            
            template_data = {"experiment_id": run_id, "model_name": model_name, "image_name": build_name, "project": project}
            applied_template = Template(open("SeldonDeploy.yaml").read())
            print(applied_template.render(template_data))
            oc.apply(applied_template.render(template_data))
            

            
            route_count = oc.selector(f"route/{build_name}").count_existing()
            print(route_count)
            if route_count == 0:
                service_name = "model-" + run_id + "-" + model_name
                while True:
                    service_count = oc.selector(f"service/{service_name}").count_existing()
                    if service_count > 0:
                        service = oc.selector(f"service/{service_name}").object()
                        print(service.name())
                        oc.oc_action(oc.cur_context(), "expose", cmd_args=["service", service.name(), "--name", service.name()])
                        break
                    else:
                        print(f"Service name does not exist {service_name}")
                        time.sleep(10)
            else:
                print(f"Route already exists {service_name}")

Note: you may need to restart the kernel to use updated packages.
Start OCP things...
OpenShift client version: 3.11.153
fmv2
1
{
    "apiVersion": "build.openshift.io/v1",
    "kind": "BuildConfig",
    "metadata": {
        "annotations": {
            "openshift.io/generated-by": "OpenShiftNewBuild"
        },
        "creationTimestamp": "2022-04-05T07:52:18Z",
        "generation": 6,
        "labels": {
            "build": "seldon-model-sademo-v6"
        },
        "managedFields": [
            {
                "apiVersion": "build.openshift.io/v1",
                "fieldsType": "FieldsV1",
                "fieldsV1": {
                    "f:metadata": {
                        "f:annotations": {
                            ".": {},
                            "f:openshift.io/generated-by": {}
                        },
                        "f:labels": {
                            ".": {},
                            "f:build": {}
                        }
              

OpenShiftPythonException: [apply returned an error: Error from server (InternalError): Internal error occurred: failed calling webhook "v1.vseldondeployment.kb.io": Post "https://seldon-webhook-service.ml-workshop.svc:443/validate-machinelearning-seldon-io-v1-seldondeployment?timeout=10s": service "seldon-webhook-service" not found]
{
    "operation": "apply",
    "status": 1,
    "actions": [
        {
            "timestamp": 1649148422766,
            "elapsed_time": 1.2812309265136719,
            "success": false,
            "status": 1,
            "verb": "apply",
            "cmd": [
                "oc",
                "apply",
                "--server=https://api.dbs-indo-1.apac-1.rht-labs.com:6443",
                "--token=**REDACTED**",
                "--namespace=fmv2",
                "-o=name",
                "-f",
                "-"
            ],
            "out": "",
            "err": "Error from server (InternalError): Internal error occurred: failed calling

In [2]:
oc.apply(applied_template.render(template_data))

OpenShiftPythonException: [apply returned an error: error: unable to recognize "STDIN": no matches for kind "List" in version "v1"]
{
    "operation": "apply",
    "status": 1,
    "actions": [
        {
            "timestamp": 1649147927664,
            "elapsed_time": 0.31944751739501953,
            "success": false,
            "status": 1,
            "verb": "apply",
            "cmd": [
                "oc",
                "apply",
                "-o=name",
                "-f",
                "-"
            ],
            "out": "",
            "err": "error: unable to recognize \"STDIN\": no matches for kind \"List\" in version \"v1\"\n",
            "references": {
                ".stack": [
                    "  File \"/usr/lib64/python3.8/runpy.py\", line 194, in _run_module_as_main\n    return _run_code(code, main_globals, None,\n",
                    "  File \"/usr/lib64/python3.8/runpy.py\", line 87, in _run_code\n    exec(code, run_globals)\n",
                 

In [ ]:
/opt/app-root/lib64/python3.8/site-packages/openshift/base_verbs.py

In [9]:
import sys  
sys.path.insert(0, '/opt/app-root/lib64/python3.8/site-packages/')

import openshift

In [11]:
applied_template.render(template_data)

'apiVersion: machinelearning.seldon.io/v1\nkind: SeldonDeployment\nmetadata:\n  labels:\n    app: seldon\n  name: model-6\nspec:\n  annotations:\n    project_name: sademo\n    deployment_version: "6"\n  #     seldon.io/engine-separate-pod: "true"\n  name: sademo-spec\n  predictors:\n    - componentSpecs:\n        - hpaSpec:\n            maxReplicas: 5\n            metrics:\n              - resource:\n                  name: cpu\n                  targetAverageUtilization: 80\n                type: Resource\n            minReplicas: 1\n          spec:\n            containers:\n              - image: "image-registry.openshift-image-registry.svc:5000/fmv2/seldon-model-sademo-v6"\n                imagePullPolicy: Always\n                name: sademotransformer\n                env:\n                  - name: MODEL_NAME\n                    value: "Transformer"\n                  - name: SERVICE_TYPE\n                    value: TRANSFORMER\n                  - name: GRPC_PORT\n             

In [12]:
openshift.apply(applied_template.render(template_data))


OpenShiftPythonException: [apply returned an error: error: unable to recognize "STDIN": no matches for kind "List" in version "v1"]
{
    "operation": "apply",
    "status": 1,
    "actions": [
        {
            "timestamp": 1649148481753,
            "elapsed_time": 0.3227119445800781,
            "success": false,
            "status": 1,
            "verb": "apply",
            "cmd": [
                "oc",
                "apply",
                "-o=name",
                "-f",
                "-"
            ],
            "out": "",
            "err": "error: unable to recognize \"STDIN\": no matches for kind \"List\" in version \"v1\"\n",
            "references": {
                ".stack": [
                    "  File \"/usr/lib64/python3.8/runpy.py\", line 194, in _run_module_as_main\n    return _run_code(code, main_globals, None,\n",
                    "  File \"/usr/lib64/python3.8/runpy.py\", line 87, in _run_code\n    exec(code, run_globals)\n",
                  